# Clasificación de muestras con una red neuronal




## 1. Lectura de las variables

<p> Se leen las variables que se han generado en el Notebook previo. De acuerdo al formato elegido, se cuenta con un fichero pickle que contiene: </p>
<ul>
    <li> <strong> x_train: </strong> Un numpy array de numpy arrays que representan las muestras que van a entrenar la red. </li>
    <li> <strong> y_train: </strong> Un numpy array de numpy arrays que representan las etiquetas (0's o 1's) que indican la clase ('señal' o 'fondo') de las muestras de <strong>x_train</strong>. </li>
    <li> <strong> x_test: </strong> Un numpy array de numpy arrays que representan las muestras que van a evaluar el rendimiento de la red. </li>
    <li> <strong> y_test: </strong> Un numpy array de numpy arrays que representan las etiquetas (0's o 1's) que indican la clase ('señal' o 'fondo') de las muestras de <strong>x_test</strong>. Estas etiquetas sirven para calcular la tasa de acierto de la red neuronal.</li>
</ul>

Las variables se extraen del fichero pickle con el comando <em>load</em>. Un ejemplo de guardar/cargar variables en formato pickle se encuentra disponible [aquí](https://wiki.python.org/moin/UsingPickle).

In [ ]:
import pickle
import numpy as np

filename = '/home/jovyan/share/Training/training_variables.p' # name of the pickle file that contains the training variables

with open(filename, 'rb') as file_:
    x_train, y_train, x_test, y_test = pickle.load(file_)

## 2. Setup del entorno keras

Para clasificar las muestras con una red neuronal es necesario construirla, compilarla y entrenarla. Para ello se trabaja con [keras](https://keras.io/), un módulo que permite construir la red añadiendo capas y seleccionando funciones de activación fácilmente.

<p> Por un lado, se importan los paquetes de keras necesarios para el entrenamiento, que incluyen entre otras cosas: </p>
<ul>
    <li> <em>Callbacks</em> para poder acceder a la información del proceso de entrenamiento (como la perdida y precisión en cada época).</li>
    <li> Módulos para incluir las capas de la red (<em>layers</em>), funciones de activación y/o implementaciónes para reducir el overfitting como el <em>Dropout</em> o regularizadores. </li>
    <li> Optimizadores como el RMSprop que servirán para definir el gradient descent.</li>
    <li> El backend sobre el cual se construye la red (tensorflow en este caso).</li>
</ul>

Adicionalmente, se incluye un módulo llamado <em>history_tools</em> que se encuentra disponible en este mismo repositorio (history_tools.py) y que permitirá procesar la información de los <em>callbacks</em> cuando la red se esté entrenando. Por un lado, define una clase <em>LossHistory</em> que se construirá a medida que el entrenamiento avance. Por otro, importa una función <em>plot_history</em> que permite representar la información gráficamente.

In [ ]:
import keras
from keras.utils import np_utils
import keras.callbacks as cb
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras import regularizers
from keras import backend as K

from history_tools import LossHistory, plot_history

## 3. Construcción de la red neuronal

<p>Antes de empezar el entrenamiento, se debe diseñar la arquitectura de la red neuronal y construirla. Esencialmente, hay que decidir:</p>
<ul>
    <li> El número de capas de la red neuronal. </li>
    <li> El número de neuronas de cada capa. </li>
    <li> Las funciones de activación de cada capa, especialmente la última. </li>
    <li> El optimizador. </li>
    <li> Otras opciones que pueden ser implementadas capa a capa para reducir el overfitting, como el <em>Dropout</em> o alguna regularización. </li>
</ul>

Keras ofrece dos maneras de construir la arquitectura de la red: el [modo secuencial](https://keras.io/getting-started/sequential-model-guide/) y la [API funcional](https://keras.io/getting-started/functional-api-guide/). Este último es más común en problemas más complejos que el presentado en este ejemplo. La red neuronal necesaria en este problema se puede construir de manera sencilla con el modo secuencial, añadiendo capas de manera lineal (por lo que el modo secuencial es el más recomendado).

<strong>Define un modelo secuencial y llámalo ```model```. Después genera una red densa que tenga la siguiente distribución de neuronas en cada una de las capas internas ````(30,20,20,10,10,10,5)````. Al ser un problema binario ¿Cuantas neuronas debe tener la última capa?  Recuerda incluir las funciones de activación recomendadas en clase para cada capa.</strong>



In [ ]:
ENTER YOUR CODE HERE

## 4. Compilación
Cuando el modelo de la red neuronal ```model``` ya está construido, con la arquitectura de capas, funciones de activación... etc definidas, el modelo debe compilarse.

En la compilación se deciden la función de perdida que se va a tratar de minimizar, el optimizador (algoritmo de optimización de dicha perdida) y la metrica que se va a utilizar para controlar el proceso de entrenamiento.

La <strong>función de perdida</strong> es un valor que pretende cuantificar de alguna manera la discrepancia entre la realidad y lo que nuestra red está prediciendo. Es decir, es una definición del error en la clasificación. Cada vez que la red termina de leer y predecir un batch, calcula la perdida resultante y trata de minimizarla. Los resultados del entrenamiento de la red dependen de la función de perdida que se utilice para definir el error. [Aquí](https://keras.io/losses/) se pueden encontrar las funciones de perdida que tiene definidas Keras.

Normalmente, en un problema de clasificación múltiple se suele utilizar la ```categorical_crossentropy```, mientras que en un problema de clasificación binario se utiliza la ```binary_crossentropy```.

El <strong>optimizador</strong> es el algoritmo de optimización que se utiliza para minimizar la pérdida y ajustar los pesos de la red. [Aquí](https://keras.io/optimizers/) se pueden encontrar los optimizadores que vienen definidos en Keras. Los más utilizados en esta clase de problemas son el ```SGD``` y el ```RMSprop```.

La <strong>métrica</strong> es una función auxiliar que se utiliza para evaluar el rendimiento de la red mientras está siendo entrenada. La elección recomendada para este modelo es la ```'accuracy'``` proporcionada por Keras.

<strong>En esta primera parte de la práctica tienes que compilar tu red neuronal. Para la primera prueba usa binary_crossentropy como función pérdida, utiliza un optimizador SGD y incluye en tus métricas el accuracy.</strong>

In [ ]:
model.compile( ENTER YOUR CODE HERE )
print('Model compiled')

## 5. Últimas consideraciones antes del entrenamiento: Formato de las muestras

En este punto la arquitectura de la red está construida y compilada. En este punto, conviene comprobar que el formato de las muestras concuerda con la arquitectura. 

En especial, una cosa que depende de manera directa con la arquitectura de la red es el formato de las etiquetas de las muestras, es decir, el formato de los vectores <strong>y_train</strong> y <strong>y_test</strong>.

Si la red es una red de clasificación binaria las etiquetas de las clases deben tener formato de ```numpy.array``` con un único elemento que indice la clase a la que pertenen (0 o 1). Es decir, los vectores tendran la forma:
```
y = numpy.array([[1],
                 [0],
                 [1],
                 [1],
                 ...,
                 [0]])
```

Por el contrario, si se interpreta el problema de clasificación como una clasificación entre múltiples clases con $N = 2$, las etiquetas deben tener formato de ```numpy.array``` con 2 elementos (un elemento por cada clase). Todas las entradas de esos arrays serán 0 excluyendo el elemento correspondiente a la clase correcta. Por ejemplo, si la clase correcta es la 1, todas los elementos del ```numpy.array``` serán iguales a cero exceptuando el elemento ```numpy.array[1]```. Es decir, para el ejemplo anterior la muestra total de etiquetas tendría la forma:
```
y = numpy.array([[0, 1],
                 [1, 0],
                 [0, 1],
                 [0, 1],
                 ...,
                 [1, 0]])
```



## 6. Entrenamiento de la red neuronal

El proceso de entrenamiento está listo para iniciarse. Por un lado, en este punto es necesario definir como va a ser ese entrenamiento (que muestras se van a utilizar, cuantas veces van a leerse, en cuantos pasos va a procesarlas la red, como se va a acceder a la información del entrenamiento... etc).


En primer lugar, es aconsejable crear un objeto de la clase ```LossHistory``` que se importó en pasos previos. Este objeto es un contenedor de <strong>información del entrenamiento </strong> que se le va a pasar como parámetro a la red. Al finalizar cada época la red va a actualizar ese objeto añadiendo información de la misma. En concreto, en este ejemplo la red va a guardar la precisión en la clasificación y el valor de la perdida al acabar cada época. Para poder actualizarse de esta manera, el objeto se le pasa com argumento de ```callbacks``` al modelo.

El <strong>número de épocas</strong> también debe ser definido, i.e. el número de veces que la red va a leer el set completo de muestras ```x_train``` que se le proporcionen para entrenar. 
<p>A la hora de elegir el número de épocas hay que alcanzar un compromiso entre dos efectos:</p>
<ul>
    <li>Si el número de épocas es muy pequeño es posible que a la red "no le de tiempo" a extraer la información y aprender lo suficiente de la muestra como para clasificar nuevos datos eficientemente. En este caso, la red clasificaría nuevas muestras de manera aleatoria.</li>
    <li> Si el número de épocas es muy grande es muy probable que aparezca <em>overfitting</em>. Lo que estaría ocurriendo es que la red ha aprendido "demasiado" de la muestra, incluidas relaciones sobre las cuales no queremos basar nuestro criterio de clasificación. Sería un caso similar al que ocurre cuando se intenta realizar un ajuste ordinario a unos puntos experimentales que tienen una cierta dispersión. Esta dispersión puede venir por la imprecisión de la medida, errores sistemáticos que afecten a los valores, fluctuaciones... etc. En este caso querriamos definir la ley experimental sobre la cual se rigen esos puntos, pero sin tener en cuenta esos efectos que no dependen de la ley, pero que aún así inevitablemente están presentes en los datos. Lo mismo ocurre con el entrenamiento de la red.</li>
</ul>

También hay que definir el <strong>tamaño del batch</strong>, es decir, el número de muestras que se van a leer en cada iteración en cada época. La red lee el batch de muestras, calcula el pérdida, optimiza los pesos y vuelve a leer un nuevo batch. El proceso se repite hasta que se terminan todas las muestras disponibles del entrenamiento y se empieza una nueva época.

El modelo ya construido, definido en el objeto ```model``` se entrena mediante la función ```fit``` que toma como argumentos:

1) Las muestras ```x_train``` y las etiquetas ```y_train```de los eventos que van a entrenar la red.

2) El número de épocas ```epochs``` y el tamaño del batch ```batch_size```.

3) El objeto de la clase ```LossHistory``` que almacena los valores de precisión y pérdida al final de cada época, y se le da como argumento ```callbacks``` al método fit.

4) Los datos de validación ```validation_data```, que van a servir para controlar el entrenamiento.

5) El nivel de ```verbose```que indica cuanta información sobre el entrenamiento se quiere mostrar en consola.

6) Un valor booleano ```shuffle``` que mezcla las muestras del entrenamiento. No es necesario si previamente en el tratamiento de los datos se han mezclado convenientemente, pero si aconsejable.


<strong>En esta primera parte de la práctica tienes que entrenar para 20 épocas con un n_batch de 512. Utiliza como dataset de validación un 10% del dataset de training, y asegurate de aplicar el shuffle para que tus muestras estén bien mezcladas. Incluye un ```callbacks``` que contenga tu history.</strong>


In [ ]:
import time
import numpy as np

print('Training model...')

history = LossHistory()

n_epochs = 
n_batch = 



start_time = time.time()


model.fit(ENTER YOUR CODE HERE)

print("Training duration : {0}".format(time.time() - start_time) + " secs")


## 7. Guardar la información del entrenamiento

Al entrenar la red neuronal sus pesos quedan fijados y la red se puede utilizar para clasificar muestras con las que la red no se haya entrenado. Guardar el modelo ya entrenado implica por un lado guardar la arquitectura (número de capas, neuronas, dropout... etc) pero también <strong>guardar los pesos de la red</strong>. De esta manera, es posible clasificar otra muestra de datos cargando de nuevo el modelo y sin necesidad de definir la estructura de la red o entrenarla nuevamente.

De cara a comparar con otros modelos y para entender si el entrenamiento se ha realizado con normalidad, <strong>la perdida y la precisión </strong> se representan gráficamente frente al número de época. A partir del estudio de esas gráficas se puede observar (si la red se ha entrenado bien) un decrecimiento de la función de pérdida y un aumento de la precisión en la clasificación a medida que pasan las épocas y la red va aprendiendo de la muestra. 

Por un lado se muestra la perdida y la precisión para las muestras de entrenamiento ```x_train```. En este caso la función de perdida 

<strong>Guarda tu modelo en formato h5 en la carpeta Models. Comprueba previamente que la carpeta exista y si no creala </strong>

In [ ]:
ENTER YOUR CODE HERE

## 8. Estudio de la pérdida y la precisión durante el entrenamiento

Para comprobar que el entrenamiento de la red ha transcurrido con normalidad, se recurre al estudio de la evolución de la pérdida y la precisión durante el entrenamiento. Se espera que por un lado la perdida disminuya, indicando así que las predicciones que da la red se van acercando más a los resultados reales a medida que aprende de la muestra. Por otro, se espera un aumento de la precisión indicando que a medida que la red aprende es capaz de clasificar las muestras con mayor tasa de éxito.

La perdida y la precisión se representan gráficamente frente al número de época. A partir del estudio de esas gráficas se pueden observar (si la red se ha entrenado bien) el decrecimiento de la pérdida y aumento de la precisión típicos que indican que la red está aprendiendo.

Por un lado, se representan la perdida y la precisión de los datos del <strong>set de entrenamiento</strong>. Generalmente, y a no ser que haya ocurrido alguna anomalía durante el entrenamiento, se observarán tanto la disminución de la perdida como el aumento de la precisión.

La evolución de la pérdida y de la precisión del <strong>set de validación</strong> es algo más compleja y es la que va dar verdadera idea de si la red está aprendiendo información eficiente para clasificar. Puesto que la red está aprendiento del set de entrenamiento, un conjunto de muestras totalmente distinto, la precisión de clasificación en las muestras de validación (muestras que nunca ha visto antes) será generalmente peor.

Problemas como el <em>overfitting</em> también se ven reflejados en la evolución de la precisión del set de validación. Cuando la red empieza a aprender patrones indeseados del set de entrenamiento y el modelo empieza a sufrir <em>overfitting</em> el incremento en la precisión en la clasificación de las muestras de validación se detiene. Los valores de la precisión comienazan a reducirse y la pérdida a crecer. Algo esperable, teniendo en cuenta que a partir de ese punto la red está basando su criterio en información no relevante. Por lo tanto, su tasa de acierto se reduce.

Para representar gráficamente la precisión y la perdida se utiza la función ```plot_history``` que esta definida en <em>history_tools.py</em> y que se importó junto con la clase ```LossHistory```. 

<strong>Antes de usar plot_history comprobar que exista la carpeta History y si no creala.</strong>



In [ ]:
ENTER YOUR CODE HERE

<strong>Ahora repite los apartados anteriores con la siguientes modificaciones:

* Usa el optimizador RMSprop
* Usa el optimizador Adam

¿Cual de los dos funciona mejor?

Quédate con el que funcione mejor de los dos. ¿Qué observas raro en la gráfica comparando la accuracy de train/validation? 

* Prueba a disminuir el learning rate

¿Qué observas ahora?

Finalmente, añadele a tu modelo final un callback de EarlyStopping y ModelCheckpoint y entrena durante 200 épocas. En el EarlyStopping utiliza como referencia val_loss y espera 5 épocas antes de decidir parar definitivamente. En el ModelChecking utiliza también val_loss y guarda el mejor modelo que hayas obtenido.

Una vez que ya tengas el modelo final elegido puedes pasar al apartado 9</strong>

## 9. Evaluar el rendimiento de la red neuronal

Se evalua el rendimiento de la red neuronal en la clasificación haciendo que clasifique la muestras del <strong>set de test</strong>. Este set está compuesto por muestras que la red nunca ha visto, por lo que la comparación entre las predicciones de la red y la clase real de los datos da una medida fiable de cuanto de eficiente es la red a la hora de discriminar entre señal y fondo.

Lo primero que se puede extraer de esta comparación son los valores de perdida y de precisión. Para ello, se utiliza la función ```evaluate``` sobre el objeto ```model``` ya entrenado. A esta función se le pasan como parámetros el ```x_test``` con las muestras de test, las etiquetas ```y_test```de esas muestras y el tamaño del batch ```batch_size```, que debe fijarse a 1 para que la red clasifica las muestras 1 a 1.

<strong>Evalua tu modelo utilizando el dataset de test. Al resultado de la evaluación llámalo ```score```.</strong>

In [ ]:
ENTER YOUR CODE HERE

<strong>Guarda ahora las predicciones que te da la red sobre el dataset de test en una variable llamada ```y_pred```. Imprime la variable para entender cual es el formato en el que se dan las predicciones.</strong>

In [ ]:
ENTER YOUR CODE HERE

### 9.1 Distribución del discriminante

La distribución de valores que toma el discriminante toma una forma muy característica dependiendo de como sea de eficiente la red neuronal clasificando las muestras de eventos del set de test.

Los valores del discriminante se representan gráficamente tanto para las muestras (reales) de señal como las muestras (reales) de fondo. De esta manera de puede estudiar la distribución de valores del discriminante para las dos clases distintas de datos.

Para la representación gráfica, se utiliza el paquete ```matplotlib.pyplot```de Python, y se construye un histograma con los valores del discriminante para los eventos de las clases 0 y 1 por separado. Se espera que el discriminante tome valores más próximos a 1 para las muestras de la clase 1 y más próximos a 0 para las muestras de la clase 0.

In [ ]:
import os
from matplotlib import pyplot as plt

# Convert the y_pred and y_test numpy matrixes to 1-dimensional numpy arrays:
discriminant = np.squeeze(np.asarray(y_pred))
true_labels = np.squeeze(np.asarray(y_test))

# Get the discriminant values for each class (0: background, 1: signal):
discriminant0 = discriminant[list(true_labels == 0)]
print(discriminant0)
discriminant1 = discriminant[list(true_labels == 1)]
print(discriminant1)

# Define the binning:
binning = np.linspace(0, 1, 51)


# Plot the discriminant distributions:
plt.clf()
plt.figure(num=None, figsize=(6, 6))
plt.subplot(111)
pdf0, bins0, patches0 = plt.hist(discriminant0, bins = binning, color = 'r', alpha = 0.3, histtype = 'stepfilled', linewidth = 1, edgecolor='r', label = 'Background')
pdf1, bins1, patches1 = plt.hist(discriminant1, bins = binning, color = 'b', alpha = 0.3, histtype = 'stepfilled', linewidth = 1, edgecolor='b', label = 'Signal')
plt.legend(loc = 'upper center')
plt.ylabel('Entries', fontsize = 12)
plt.xlabel('DNN discriminant', fontsize = 12)
if not os.path.exists('Results/'): os.makedirs('Results/')
plt.savefig('Results/Discriminant_distribution.png', dpi = 600)


### 9.2 Curva ROC

En un problema de clasificación el objetivo es maximizar el número de aciertos en la clasificación minimizando a su vez el número de errores. Es decir, se intenta identificar el mayor número posible de eventos de señal verdaderos evitando clasificar eventos de fondo verdaderos como eventos de señal. 

<p>Se definen, en este contexto:</p>
<ul>
    <li> Verdaderos Positivos <strong>VP</strong>: Eventos de señal que son identificados como eventos de señal </li>
    <li> Falsos Positivos <strong>FP</strong>: Eventos de fondo que son identificados (erróneamente) como eventos de señal </li>
    <li> Falsos Negativos <strong>FN</strong>: Eventos de señal que son identificados (erróneamente) como eventos de fondo </li>
    <li> Verdaderos Negativos <strong>VN</strong>: Eventos de fondo que son identificados como eventos de fondo </li>
</ul>

La <strong>sensibilidad</strong> de la red neuronal se define como la razón de verdaderos positivos <strong>VPR</strong>. Esto es, la razón de verdaderos positivos con respecto al número de muestras de señal (eventos de clase 1) que hay en el set de test.

Por otro lado, la <strong>razón de falsos positivos</strong> <strong>FPR</strong> se define como el ratio entre el número de falsos positivos y el número de muestras de fondo (eventos de clase 0) que hay en el set de test.

Los valores de la VPR y FPR dependen del criterio que se utilice con el discriminante para clasificar una muestra como señal o como fondo. Tras obtener las distribuciones del discriminante se puede interpretar libremente sobre que punto de corte las muestras son consideradas señal. Por ejemplo, se podría interpretar que toda muestra con discriminante mayor que 0.5 sea considerada como señal, y que toda muestra por debajo de dicho valor sea fondo. Sin embargo, también podríamos elegir ese corte como 0.3, 0.4... etc o cualquier otro valor de tal manera que la relación entre la VPR y FPR se optimizase.

La <strong>curva ROC</strong> es una representación gráfica de la sensibilidad (VPR) frente a la FPR para un sistema clasificador binario según se varía el umbral de discriminación (de 0 a 1). En la Figura 9.1 se muestra un ejemplo de dicha curva.

Para una clasificación totalmente aleatoria se esperaría que las razones de verdaderos positivos y falsos positivos fuesen exactamente iguales, ya que la probabilidad de acierto de la red neuronal sería del 50%. Por lo tanto, una curva ROC de una clasificación aleatoria se representaría con una recta de pendiente 1 en el espacio ROC, esto es VPR = FPR. Una curva ROC de un ejemplo de clasificación general tomaría una forma convexa sobre dicha recta, siendo más alto el área contenida bajo la curva <strong>AUC</strong> cuanto mejor sea el rendimiento de la red. Una clasificación perfecta se correspondería con un punto en el espacio ROC de (FPR, VPR) = (0, 1). Curvas que se encuentren por debajo de la recta VPR = FPR son ejemplos de malas clasificaciones en las cuales la red está utilizando un criterio opuesto al que debería utilizarse.

___
<img src="https://upload.wikimedia.org/wikipedia/commons/3/36/ROC_space-2.png" width="40%">

Figura 9.1. Ejemplo de una curva ROC
___

Para la implementación de la curva ROC en el código se hace uso del paquete de Python ```sklearn``` cuya documentación puede encontrarse [aquí](https://scikit-learn.org/stable/). Dentro del módulo ```sklearn.metrics``` se define la función ```roc_curve``` que toma como argumentos las predicciones de la red ```y_pred``` y las etiquetas de clases verdaderas ```y_test``` del set de test. La función calcula la VPR (```tpr```) y la FPR (```fpr```) para cada valor del punto de corte del discriminante (```thresholds```) y devuelve esos 3 valores contenidos en objetos ```numpy.array```. Un ejemplo de la sintaxis sería:


```fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)```

A partir de las VPR y FPR es posible calcular el área bajo la curva ROC. Esto puede hacerse fácilmente con otra función también implementada en ```sklearn```,  llamada ```metrics.auc```. La implementación sería:

```auc = metrics.auc(fpr, tpr)```

Normalmente se suele tener en cuenta el siguiente criterio para evaluar la AUC:

```.90-1 = excellent (A)```
```.80-.90 = good (B)```
```.70-.80 = fair (C)```
```.60-.70 = poor (D)```
```.50-.60 = fail (F)```

In [ ]:
import os
from matplotlib import pyplot as plt
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
auc = metrics.auc(fpr, tpr)

plt.clf()
plt.figure(num=None, figsize=(6, 6))
plt.subplot(111)
plt.plot(fpr, tpr, color = 'r', label = "ROC curve")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label = "Random guess")
plt.legend(loc = "lower right")
plt.xlabel('False Positive rate', fontsize = 12)
plt.ylabel('True Positive rate', fontsize = 12)
plt.text(0.68, 0.1, 'AUC: %.3f' % auc)
plt.savefig('Results/ROC.png', dpi =  600)


#### Optativo y no explicado (Youden index, cut value):
Youden_index = tpr - fpr
i_max = np.argmax(Youden_index)
cut_value = thresholds[i_max]

print("The optimal cut value is: " + str(cut_value))

### 9.3 Matriz de confusión

Finalmente y una vez que se ha decidido el punto de corte del discriminante, los resultados de la clasificación para la red neuronal que ha sido entrenada pueden expresarse como una tabla de contingencia, más conocida como <strong>matriz de confusión</strong>.

Cada columna de la matriz representa los valores predichos por la red, mientras que las filas son los valores reales de las clases. En este caso, solo se clasifica en dos clases: 0, fondo, y 1, señal. Por lo tanto, la matriz tendrá dimensión 2x2. De acuerdo a esta definición los VN se cuentan en el cuadrante (0,0), los FP en el cuadrante (0,1), los FN en el (1,0) y los VP en el (1,1).

In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
from results_tools import plot_confusion_matrix

print("-> Computing confusion matrix...")

y_test_confusion = true_labels
y_pred_confusion = np.zeros(len(discriminant))

for i in range(0, len(discriminant)):
    if discriminant[i] > cut_value: # threshold
        y_pred_confusion[i] = 1
    else:
        y_pred_confusion[i] = 0
        

cnf_matrix = confusion_matrix(y_test_confusion, y_pred_confusion)


plt.figure()
plot_confusion_matrix(cm=cnf_matrix, classes=['Background (0)', 'Signal (1)'], normalize=True, title='Normalized confusion matrix')

if not os.path.exists('Results/'): os.makedirs('Results/')
plt.savefig("Results/ConfusionMatrix.png", dpi = 600)